In [1]:
sys.path.append('C:/Users/nyasha/Desktop/Masters-Nyasha/Models')

import sys

import torch
import numpy as np
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from DatasetCreator import HAVSDataset

from CAE import *
from training_functions import *

In [2]:
batch_size = 128
num_epochs = 25

In [3]:
# Extracting the training, validation and testing data
compressed_file_path = "C:/Users/nyasha/Desktop/Masters-Nyasha/Processed Data/processed_data.npz"
x_train, x_test, x_val, y_train, y_test, y_val = get_train_test_data(compressed_file_path)

In [4]:
# Check if files have been imported correctly
print(x_train.shape)  
print(x_test.shape)
print(x_val.shape)
print(y_train.shape) 
print(y_test.shape)
print(y_val.shape)

(11035, 128, 44)
(3679, 128, 44)
(3679, 128, 44)
(11035,)
(3679,)
(3679,)


In [5]:
np.amax(x_train)

1.0

In [6]:
# Define the transforms
transform = transforms.Compose(
    [
     transforms.ToTensor()
    ])
# Create the datasets
train_dataset = HAVSDataset(x_train, y_train, transform=transform)
val_dataset = HAVSDataset(x_val, y_val, transform=transform)
test_dataset = HAVSDataset(x_test, y_test, transform=transform)

In [7]:
# Creating the data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [9]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 32, 3)
        self.dropout = nn.Dropout(0.25)
        self.fc1 = nn.Linear(32 * 30 * 9, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 6)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 30 * 9)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [24]:
class ConvNet1(nn.Module):
    def __init__(self):
        super(ConvNet1, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 10, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(10, 20, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(20,20, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc1 = nn.Sequential(
            nn.Linear(20 * 16 *8, 128),
            nn.ReLU())
        self.fc2 = nn.Linear(128, 4)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [10]:
# Select a model from ../Models
model_1  = Net() #ConvNet1()

In [36]:
data = iter(train_loader)
images, _ = next(data)

In [11]:
device = get_device() 
model_on_device = model_1.to(device) # Move model to the current device

Running on the GPU


In [29]:
images = images.to(device, dtype=torch.float)

In [30]:
x = model_1.layer1(images)
print(x.shape)
x = model_1.layer2(x)
print(x.shape)
x = model_1.layer3(x)
print(x.shape)

ModuleAttributeError: 'Net' object has no attribute 'layer1'

In [31]:
x = model_1.pool(F.relu(model_1.conv1(images)))
x = model_1.pool(F.relu(model_1.conv2(x)))
x.shape

torch.Size([128, 32, 30, 9])

In [12]:
class_weights = get_class_weights(y_train, device)
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.SGD(model_on_device.parameters(), lr=0.005, momentum=0.9)

C:\Users\nyasha\anaconda3\envs\masters\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=['2_walking' 'clutter/noise' 'running' 'sphere_swing' 'vehicle' 'walking'], y=['walking' 'vehicle' 'sphere_swing' ... 'sphere_swing' 'walking' 'vehicle'] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [13]:
class_weights

tensor([1.4124, 0.8343, 2.2706, 0.6050], device='cuda:0')

In [13]:
model_on_device

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=13440, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=4, bias=True)
)

In [13]:
train_model(50, train_loader, val_loader, criterion, optimizer, device, model_on_device)

Epoch: 1/50 	 Training Loss: 1.2708, Accuracy: 20.52, Testing Loss: 1.7914, Accuracy: 23.97
Epoch: 2/50 	 Training Loss: 1.2699, Accuracy: 21.38, Testing Loss: 1.7909, Accuracy: 13.54
Epoch: 3/50 	 Training Loss: 1.2697, Accuracy: 17.49, Testing Loss: 1.7896, Accuracy: 23.97
Epoch: 4/50 	 Training Loss: 1.2684, Accuracy: 19.61, Testing Loss: 1.7875, Accuracy: 18.10
Epoch: 5/50 	 Training Loss: 1.2668, Accuracy: 22.33, Testing Loss: 1.7833, Accuracy: 15.77
Epoch: 6/50 	 Training Loss: 1.2596, Accuracy: 24.29, Testing Loss: 1.7699, Accuracy: 17.04
Epoch: 7/50 	 Training Loss: 1.2265, Accuracy: 28.26, Testing Loss: 1.6651, Accuracy: 28.81
Epoch: 8/50 	 Training Loss: 1.1966, Accuracy: 29.66, Testing Loss: 1.6718, Accuracy: 38.46
Epoch: 9/50 	 Training Loss: 1.1967, Accuracy: 28.33, Testing Loss: 1.7516, Accuracy: 23.16
Epoch: 10/50 	 Training Loss: 1.1486, Accuracy: 36.14, Testing Loss: 1.5818, Accuracy: 28.21
Epoch: 11/50 	 Training Loss: 0.9904, Accuracy: 40.67, Testing Loss: 1.3508, Ac

In [14]:
# Evaluate model performance on test data
evaluate_model(test_loader, device, model_on_device, criterion)

(0.4445792775729607, 83.31068225061158)